In [1]:
import geopandas as gpd
import pandas as pd

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import pandas as pd
import geopandas as gpd
import psycopg2 as pg

engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")

In [6]:
df_buildings = gpd.read_postgis("select physical_object_id as id, is_living, geometry as geom from all_buildings where city_id=1 and administrative_unit='Василеостровский'", con=engine)
df_buildings

/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,id,is_living,geom
0,32195,False,"POLYGON ((30.18993 59.95255, 30.18997 59.95262..."
1,32232,False,"POLYGON ((30.19049 59.95234, 30.19052 59.95240..."
2,32323,False,"POLYGON ((30.19178 59.94517, 30.19224 59.94595..."
3,32370,False,"POLYGON ((30.19232 59.95181, 30.19255 59.95219..."
4,32428,False,"POLYGON ((30.19306 59.94707, 30.19316 59.94725..."
...,...,...,...
3744,34479,False,"POLYGON ((30.21352 59.95188, 30.21360 59.95200..."
3745,31701,False,"POLYGON ((30.18337 59.94394, 30.18368 59.94445..."
3746,31746,False,"POLYGON ((30.18384 59.95347, 30.18387 59.95351..."
3747,32016,False,"POLYGON ((30.18749 59.95224, 30.18780 59.95276..."


In [7]:
# df_buildings = gpd.read_postgis('select physical_object_id as id, is_living, geometry as geom from all_buildings where city_id=1', con=engine)
# df_buildings

In [5]:
df_buildings.to_parquet('DF_BUILDINGS_POLY.parquet')

In [8]:
# gdf = gpd.read_parquet('/home/gk/jupyter/DF_BUILDINGS.parquet')

In [6]:
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from shapely.ops import cascaded_union
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
from geovoronoi import voronoi_regions_from_coords, points_to_coords

In [7]:
from blocks_getter import BlocksModel

model = BlocksModel(city_crs=32636, city_admin_level=5, city_name='Санкт-Петербург')
model._get_city_geometry()

2023-04-17 20:50:00.053 | INFO     | blocks_getter:_get_city_geometry:103 - City geometry is not provided. Getting water geometry from OSM via overpass turbo
/home/gk/.local/lib/python3.8/site-packages/osm2geojson/main.py:514: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in merged_line:
2023-04-17 20:50:02.757 | INFO     | blocks_getter:_get_city_geometry:117 - Got city geometry


In [8]:
boundary = model.city_geometry

In [10]:
df_buildings.dropna(subset='geom', inplace=True)

In [ ]:
df_buildings = df_buildings.

In [12]:
df_buildings = df_buildings.to_crs(32636)

In [13]:
b = gpd.sjoin(df_buildings, boundary, predicate='intersects')

In [14]:
boundary_shape = cascaded_union(boundary.geometry)
coords = points_to_coords(b.geometry)

/tmp/ipykernel_2139970/1585934383.py:1: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  boundary_shape = cascaded_union(boundary.geometry)


NotImplementedError: Component rings have coordinate sequences, but the polygon does not

In [ ]:
poly_shapes, pts = voronoi_regions_from_coords(coords, boundary_shape)
# poly_shapes, pts, poly_to_pt_assignments

In [ ]:
gs = gpd.GeoSeries(poly_shapes)
gdf = gpd.GeoDataFrame(gs, columns=["geometry"])
gdf.to_file("voronoi_2.geojson")

In [ ]:
import pickle

with open('poly_shapes_2.pkl', 'wb') as f:
    pickle.dump(poly_shapes, f)

with open('pts_2.pkl', 'wb') as f:
    pickle.dump(pts, f)

In [ ]:
fig, ax = subplot_for_map()
plot_voronoi_polys_with_points_in_area(ax, boundary_shape, poly_shapes, pts, poly_to_pt_assignments)
ax.set_title('Voronoi regions of Schools in Uppsala')
plt.tight_layout()
plt.show()